In [1]:
import time
import numpy as np
import pandas as pd
import time
import pickle
import sys
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get('https:/1xbet.whoscored.com/')
driver.implicitly_wait(3)
driver.close()

In [3]:
def crawling_player_statistics(URL):
    """
    crawling player statistics of certain league and season.
    
    Args :
        URL : player statistics webpage URL
        
    return :
        player statistics (dataframe)
    
    """
    url = str(URL)
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(URL)    
    
    try:
        driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]/span').click()
    except NoSuchElementException:
        pass

    driver.find_element(By.XPATH, '//*[@id="apps"]/dd[2]/a').click()
    time.sleep(5)
    page = driver.find_elements(By.CSS_SELECTOR, '#statistics-paging-summary > div > dl.listbox.right > dt > b')[0].get_attribute("textContent").split("\\")[0]
    page = int(page.split('/')[1].split(' ')[0])
    player_statistics_df = pd.DataFrame(columns=[
        "player_name", "team_number", "team_name", "Apps", "Mins", "Goals",
        "Assists", "Yel", "Red", "SpG","PS%", "AerialsWon","MoM","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
      
        for i in range(page):
            if i != 0:
                driver.find_element(By.ID, 'next').click()
                time.sleep(1.5)

            temp_df = pd.DataFrame(columns=[
                "player_name", "team_number", "team_name", "Apps", "Mins", "Goals",
                "Assists", "Yel", "Red", "SpG","PS%", "AerialsWon","MoM","Rating"])
            elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")

            for element in elements:
                player_statistics_dict = { 
                    "player_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].
                    find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0],
                    "team_number": element.find_elements(By.CSS_SELECTOR, "td")[0].
                    find_elements(By.CSS_SELECTOR, "a")[1].get_attribute("href").split("/")[4],
                    "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, ("a"))[1].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0].split(",")[0],
                    "Apps": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
                    "Mins": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
                    "Goals": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0],
                    "Assists": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
                    "Yel": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0], 
                    "Red": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
                    "SpG": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
                    "PS%": element.find_elements(By.CSS_SELECTOR, "td")[9].get_attribute("textContent").split("\t")[0],
                    "AerialsWon": element.find_elements(By.CSS_SELECTOR, "td")[10].get_attribute("textContent").split("\t")[0],
                    "MoM": element.find_elements(By.CSS_SELECTOR, "td")[11].get_attribute("textContent").split("\t")[0],
                    "Rating": element.find_elements(By.CSS_SELECTOR, "td")[12].get_attribute("textContent").split("\t")[0],
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict

            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
            
    driver.close()
    player_statistics_df.replace("-", "0", inplace=True)
    return(player_statistics_df)
    

In [4]:
def crawling_player_statistics_defensive(URL):
    """
    crawling player statistics(defensive) of certain league and season.
    
    Args :
        URL : player statistics webpage URL
        
    return :
        player statistics (dataframe)
    
    """
    url = str(URL)
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)   

    time.sleep(1.5) 

    try:
        driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]/span').click()
    except NoSuchElementException:
        pass
    
    time.sleep(1.5)
    
    driver.find_element(By.LINK_TEXT, "Defensive").click()

    
    time.sleep(1.5)

    
    driver.find_element(By.XPATH, '/html/body/div[5]/div[5]/div[3]/div[1]/div[2]/dl[2]/dd[2]/a').click()


    time.sleep(2.5)
    page = driver.find_elements(By.CSS_SELECTOR, '#statistics-paging-defensive > div > dl.listbox.right > dt > b')[0].get_attribute("textContent")
    page = int(page.split('/')[1].split(' ')[0])
    
    player_statistics_df = pd.DataFrame(columns=[
            "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Tackles",
            "Inter", "Fouls", "Offsides", "Clear","Drb", "Blocks","OwnG","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element(By.LINK_TEXT, 'next').click()
                time.sleep(1.5)

            temp_df = pd.DataFrame(columns=[
                "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Tackles",
                "Inter", "Fouls", "Offsides", "Clear","Drb", "Blocks","OwnG","Rating"])

            elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")[10:]

            for element in elements:
                player_statistics_dict = { 
                    "player_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0],
                    "team_number": element.find_elements(By.CSS_SELECTOR, "td")[0].
                    find_elements(By.CSS_SELECTOR, "a")[1].get_attribute("href").split("/")[4],
                    "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[1].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0].split(",")[0],
                    "age": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[4].get_attribute("textContent").split("\t")[0],
                    "position": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[5].get_attribute("textContent").split("\t")[0][2:],
                    "Apps": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
                    "Mins": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
                    "Tackles": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0],
                    "Inter": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
                    "Fouls": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0], 
                    "Offsides": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
                    "Clear": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
                    "Drb": element.find_elements(By.CSS_SELECTOR, "td")[9].get_attribute("textContent").split("\t")[0],
                    "Blocks": element.find_elements(By.CSS_SELECTOR, "td")[10].get_attribute("textContent").split("\t")[0],
                    "OwnG": element.find_elements(By.CSS_SELECTOR, "td")[11].get_attribute("textContent").split("\t")[0],
                    "Rating": element.find_elements(By.CSS_SELECTOR, "td")[12].get_attribute("textContent").split("\t")[0],
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict


            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
        
    driver.close()
    
    player_statistics_df.replace("-", "0", inplace=True)
    return(player_statistics_df)
    

In [5]:
def crawling_player_statistics_offensive(URL):
    """
    crawling player statistics(offensive) of certain league and season.
    
    Args :
        URL : player statistics webpage URL
        
    return :
        player statistics (dataframe)
    
    """
    url = str(URL)
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)   

    time.sleep(1.5) 
    
    try:
        driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]/span').click()
    except NoSuchElementException:
        pass
    

    time.sleep(1.5)
    
    driver.find_element(By.LINK_TEXT, "Offensive").click()

    
    time.sleep(1.5)

    
    driver.find_element(By.LINK_TEXT, "All players").click()
    
    time.sleep(1.5)

    page = driver.find_elements(By.CSS_SELECTOR, '#statistics-paging-offensive > div > dl.listbox.right > dt > b')[0].get_attribute("textContent").split("\t")[0]
    page = int(page.split('/')[1].split(' ')[0])
    
    player_statistics_df = pd.DataFrame(columns=[
            "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Goals",
            "Assists", "SpG", "KeyP", "Dribble","Fouled", "Off","Disp","UnsTch","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element(By.LINK_TEXT, 'next').click()
                time.sleep(1.5)

            temp_df = pd.DataFrame(columns=[
                "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Goals",
                "Assists", "SpG", "KeyP", "Dribble","Fouled", "Off","Disp","UnsTch","Rating"])

            elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")[10:]

            for element in elements:
                player_statistics_dict = { 
                    "player_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].
                    find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0],
                    "team_number": element.find_elements(By.CSS_SELECTOR, "td")[0].
                    find_elements(By.CSS_SELECTOR, "a")[1].get_attribute("href").split("/")[4],
                    "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR,"a")[1].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0].split(",")[0],
                    "age": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR,"span")[4].get_attribute("textContent").split("\t")[0],
                    "position": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[5].get_attribute("textContent").split("\t")[0][2:],
                    "Apps": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
                    "Mins": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
                    "Goals": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0],
                    "Assists": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
                    "SpG": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0], 
                    "KeyP": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
                    "Dribble": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
                    "Fouled": element.find_elements(By.CSS_SELECTOR, "td")[9].get_attribute("textContent").split("\t")[0],
                    "Off": element.find_elements(By.CSS_SELECTOR, "td")[10].get_attribute("textContent").split("\t")[0],
                    "Disp": element.find_elements(By.CSS_SELECTOR, "td")[11].get_attribute("textContent").split("\t")[0],
                    "UnsTch": element.find_elements(By.CSS_SELECTOR, "td")[12].get_attribute("textContent").split("\t")[0],
                    "Rating": element.find_elements(By.CSS_SELECTOR, "td")[13].get_attribute("textContent").split("\t")[0],
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict


            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
        
    driver.close()

    player_statistics_df.replace("-", "0", inplace=True)
    return(player_statistics_df)

In [6]:
def crawling_player_statistics_passing(URL):

    url = str(URL)
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)   

    time.sleep(1.5) 
    
    try:
        driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]/span').click()
    except NoSuchElementException:
        pass

    time.sleep(1.5)
    
    driver.find_element(By.LINK_TEXT, "Passing").click()

    
    time.sleep(1.5)

    
    driver.find_element(By.LINK_TEXT, "All players").click()


    time.sleep(1.5)
    page = driver.find_elements(By.CSS_SELECTOR, '#statistics-paging-passing > div > dl.listbox.right > dt > b')[0].get_attribute("textContent")
    page = int(page.split('/')[1].split(' ')[0])
    
    player_statistics_df = pd.DataFrame(columns=[
            "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Assists",
            "KeyP", "AvgP","PS%", "Crosses","LongB","ThrB","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element(By.LINK_TEXT, 'next').click()
                time.sleep(1.5)

            temp_df = pd.DataFrame(columns=[
                "player_name", "team_number", "team_name", "age","position", "Apps", "Mins","Assists",
                "KeyP", "AvgP","PS%", "Crosses","LongB","ThrB","Rating"])

            elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")[10:]

            for element in elements:
                player_statistics_dict = { 
                    "player_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].
                    find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0],
                    "team_number": element.find_elements(By.CSS_SELECTOR, "td")[0].
                    find_elements(By.CSS_SELECTOR, "a")[1].get_attribute("href").split("/")[4],
                    "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[1].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0].split(",")[0],
                    "age": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[4].get_attribute("textContent").split("\t")[0],
                    "position": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[5].get_attribute("textContent").split("\t")[0][2:],
                    "Apps": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
                    "Mins": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
                    "Assists": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0],
                    "KeyP": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
                    "AvgP": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0], 
                    "PS%": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
                    "Crosses": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
                    "LongB": element.find_elements(By.CSS_SELECTOR, "td")[9].get_attribute("textContent").split("\t")[0],
                    "ThrB": element.find_elements(By.CSS_SELECTOR, "td")[10].get_attribute("textContent").split("\t")[0],
                    "Rating": element.find_elements(By.CSS_SELECTOR, "td")[11].get_attribute("textContent").split("\t")[0],
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict


            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
        
    driver.close()
    player_statistics_df.replace("-", "0", inplace=True)

    return(player_statistics_df)

# crwaling player stats at once

In [7]:
def crwaling_player_stats_at_once(URL):
    stats_summary = crawling_player_statistics(URL)
    print('player stats summary : done')
    stats_defensive = crawling_player_statistics_defensive(URL)
    print('player stats defensive : done')
    stats_offensive = crawling_player_statistics_offensive(URL)
    print('player stats offensive : done')
    stats_passing = crawling_player_statistics_passing(URL)
    print('player stats passing : done')
    player_stats = pd.merge(pd.merge(stats_passing, stats_summary), 
                            pd.merge(stats_offensive, stats_defensive))
    return(player_stats)

In [11]:
Premier = [
'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/6335/Stages/13796/PlayerStatistics/England-Premier-League-2016-2017'
'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/6829/Stages/15151/PlayerStatistics/England-Premier-League-2017-2018'
'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/7361/Stages/16368/PlayerStatistics/England-Premier-League-2018-2019'
'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/7811/Stages/17590/PlayerStatistics/England-Premier-League-2019-2020'
'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/8228/Stages/18685/PlayerStatistics/England-Premier-League-2020-2021'
'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/8618/Stages/19793/PlayerStatistics/England-Premier-League-2021-2022'
]

Championship = [
  'https://1xbet.whoscored.com/Regions/252/Tournaments/7/Seasons/6365/Stages/13832/PlayerStatistics/England-Championship-2016-2017',
  'https://1xbet.whoscored.com/Regions/252/Tournaments/7/Seasons/6848/Stages/15177/PlayerStatistics/England-Championship-2017-2018',
  'https://1xbet.whoscored.com/Regions/252/Tournaments/7/Seasons/7379/Stages/16389/PlayerStatistics/England-Championship-2018-2019',
  'https://1xbet.whoscored.com/Regions/252/Tournaments/7/Seasons/7840/Stages/17629/PlayerStatistics/England-Championship-2019-2020',
  'https://1xbet.whoscored.com/Regions/252/Tournaments/7/Seasons/8304/Stages/18825/PlayerStatistics/England-Championship-2020-2021',
  'https://1xbet.whoscored.com/Regions/252/Tournaments/7/Seasons/8619/Stages/19794/PlayerStatistics/England-Championship-2021-2022'


]

SerieA = [
'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/6461/Stages/14014/PlayerStatistics/Italy-Serie-A-2016-2017',
'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/6974/Stages/15404/PlayerStatistics/Italy-Serie-A-2017-2018',
'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/7468/Stages/16548/PlayerStatistics/Italy-Serie-A-2018-2019',
'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/7928/Stages/17835/PlayerStatistics/Italy-Serie-A-2019-2020',
'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/8330/Stages/18873/PlayerStatistics/Italy-Serie-A-2020-2021',
'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/8735/Stages/19982/PlayerStatistics/Italy-Serie-A-2021-2022'
]

Laliga = [
'https://1xbet.whoscored.com/Regions/206/Tournaments/4/Seasons/6436/Stages/13955/PlayerStatistics/Spain-LaLiga-2016-2017',
'https://1xbet.whoscored.com/Regions/206/Tournaments/4/Seasons/6960/Stages/15375/PlayerStatistics/Spain-LaLiga-2017-2018',
'https://1xbet.whoscored.com/Regions/206/Tournaments/4/Seasons/7466/Stages/16546/PlayerStatistics/Spain-LaLiga-2018-2019',
'https://1xbet.whoscored.com/Regions/206/Tournaments/4/Seasons/7889/Stages/17702/PlayerStatistics/Spain-LaLiga-2019-2020',
'https://1xbet.whoscored.com/Regions/206/Tournaments/4/Seasons/8321/Stages/18851/PlayerStatistics/Spain-LaLiga-2020-2021',
'https://1xbet.whoscored.com/Regions/206/Tournaments/4/Seasons/8681/Stages/19895/PlayerStatistics/Spain-LaLiga-2021-2022'
]

Bundesliga = [
'https://1xbet.whoscored.com/Regions/81/Tournaments/3/Seasons/6392/Stages/13872/PlayerStatistics/Germany-Bundesliga-2016-2017',
'https://1xbet.whoscored.com/Regions/81/Tournaments/3/Seasons/6902/Stages/15243/PlayerStatistics/Germany-Bundesliga-2017-2018',
'https://1xbet.whoscored.com/Regions/81/Tournaments/3/Seasons/7405/Stages/16427/PlayerStatistics/Germany-Bundesliga-2018-2019',
'https://1xbet.whoscored.com/Regions/81/Tournaments/3/Seasons/7872/Stages/17682/PlayerStatistics/Germany-Bundesliga-2019-2020',
'https://1xbet.whoscored.com/Regions/81/Tournaments/3/Seasons/8279/Stages/18762/PlayerStatistics/Germany-Bundesliga-2020-2021',
'https://1xbet.whoscored.com/Regions/81/Tournaments/3/Seasons/8667/Stages/19862/PlayerStatistics/Germany-Bundesliga-2021-2022'
]

Ligue1 = [
'https://1xbet.whoscored.com/Regions/74/Tournaments/22/Seasons/6318/Stages/13768/PlayerStatistics/France-Ligue-1-2016-2017',
'https://1xbet.whoscored.com/Regions/74/Tournaments/22/Seasons/6833/Stages/15155/PlayerStatistics/France-Ligue-1-2017-2018',
'https://1xbet.whoscored.com/Regions/74/Tournaments/22/Seasons/7344/Stages/16348/PlayerStatistics/France-Ligue-1-2018-2019',
'https://1xbet.whoscored.com/Regions/74/Tournaments/22/Seasons/7814/Stages/17593/PlayerStatistics/France-Ligue-1-2019-2020',
'https://1xbet.whoscored.com/Regions/74/Tournaments/22/Seasons/8185/Stages/18594/PlayerStatistics/France-Ligue-1-2020-2021',
'https://1xbet.whoscored.com/Regions/74/Tournaments/22/Seasons/8671/Stages/19866/PlayerStatistics/France-Ligue-1-2021-2022' 
]

Portugal = [
'https://1xbet.whoscored.com/Regions/177/Tournaments/21/Seasons/6438/Stages/13957/PlayerStatistics/Portugal-Liga-2016-2017',
'https://1xbet.whoscored.com/Regions/177/Tournaments/21/Seasons/6933/Stages/15323/PlayerStatistics/Portugal-Liga-2017-2018',
'https://1xbet.whoscored.com/Regions/177/Tournaments/21/Seasons/7429/Stages/16461/PlayerStatistics/Portugal-Liga-2018-2019',
'https://1xbet.whoscored.com/Regions/177/Tournaments/21/Seasons/7893/Stages/17710/PlayerStatistics/Portugal-Liga-2019-2020',
'https://1xbet.whoscored.com/Regions/177/Tournaments/21/Seasons/8315/Stages/18842/PlayerStatistics/Portugal-Liga-2020-2021',
'https://1xbet.whoscored.com/Regions/177/Tournaments/21/Seasons/8714/Stages/19947/PlayerStatistics/Portugal-Liga-2021-2022'
]

Eredivisie = [
  #'https://1xbet.whoscored.com/Regions/155/Tournaments/13/Seasons/6331/Stages/13792/PlayerStatistics/Netherlands-Eredivisie-2016-2017',
  #'https://1xbet.whoscored.com/Regions/155/Tournaments/13/Seasons/6826/Stages/15148/PlayerStatistics/Netherlands-Eredivisie-2017-2018',
  #'https://1xbet.whoscored.com/Regions/155/Tournaments/13/Seasons/7354/Stages/16360/PlayerStatistics/Netherlands-Eredivisie-2018-2019',
  'https://1xbet.whoscored.com/Regions/155/Tournaments/13/Seasons/7815/Stages/17594/PlayerStatistics/Netherlands-Eredivisie-2019-2020',
  'https://1xbet.whoscored.com/Regions/155/Tournaments/13/Seasons/8187/Stages/18596/PlayerStatistics/Netherlands-Eredivisie-2020-2021',
  'https://1xbet.whoscored.com/Regions/155/Tournaments/13/Seasons/8625/Stages/19802/PlayerStatistics/Netherlands-Eredivisie-2021-2022'
]

Belgique = [
  'https://1xbet.whoscored.com/Regions/22/Tournaments/18/Seasons/8197/Stages/18620/PlayerStatistics/Belgium-Jupiler-Pro-League-2020-2021',
  'https://1xbet.whoscored.com/Regions/22/Tournaments/18/Seasons/8627/Stages/19808/PlayerStatistics/Belgium-Jupiler-Pro-League-2021-2022'
]

Argentina = [
  "https://1xbet.whoscored.com/Regions/11/Tournaments/68/Seasons/6484/Stages/14066/PlayerStatistics/Argentina-Liga-Profesional-2016-2017",
  "https://1xbet.whoscored.com/Regions/11/Tournaments/68/Seasons/6995/Stages/15436/PlayerStatistics/Argentina-Liga-Profesional-2017-2018",
  "https://1xbet.whoscored.com/Regions/11/Tournaments/68/Seasons/7460/Stages/16537/PlayerStatistics/Argentina-Liga-Profesional-2018-2019",
  "https://1xbet.whoscored.com/Regions/11/Tournaments/68/Seasons/7905/Stages/17737/PlayerStatistics/Argentina-Liga-Profesional-2019-2020",
  "https://1xbet.whoscored.com/Regions/11/Tournaments/68/Seasons/8679/Stages/19893/PlayerStatistics/Argentina-Liga-Profesional-2021",
  "https://1xbet.whoscored.com/Regions/11/Tournaments/68/Seasons/9081/Stages/20951/PlayerStatistics/Argentina-Liga-Profesional-2022"
]

Brazil = [
  "https://1xbet.whoscored.com/Regions/31/Tournaments/95/Seasons/6700/Stages/14746/PlayerStatistics/Brazil-Brasileir%C3%A3o-2017",
  "https://1xbet.whoscored.com/Regions/31/Tournaments/95/Seasons/7243/Stages/15996/PlayerStatistics/Brazil-Brasileir%C3%A3o-2018",
  "https://1xbet.whoscored.com/Regions/31/Tournaments/95/Seasons/7683/Stages/17175/PlayerStatistics/Brazil-Brasileir%C3%A3o-2019",
  "https://1xbet.whoscored.com/Regions/31/Tournaments/95/Seasons/8158/Stages/18472/PlayerStatistics/Brazil-Brasileir%C3%A3o-2020",
  "https://1xbet.whoscored.com/Regions/31/Tournaments/95/Seasons/8555/Stages/19551/PlayerStatistics/Brazil-Brasileir%C3%A3o-2021",
  "https://1xbet.whoscored.com/Regions/31/Tournaments/95/Seasons/8984/Stages/20566/PlayerStatistics/Brazil-Brasileir%C3%A3o-2022"
]


for i in Eredivisie:
  Data = crwaling_player_stats_at_once(i)
  Data.to_csv('../Data/Whoscored/'+i.split('/')[12]+'.csv')

page 49: 100%|██████████| 49/49 [04:30<00:00,  5.51s/it]
player stats summary : done
page 49: 100%|██████████| 49/49 [04:35<00:00,  5.63s/it]
player stats defensive : done
page 49: 100%|██████████| 49/49 [04:51<00:00,  5.96s/it]
player stats offensive : done
page 49: 100%|██████████| 49/49 [05:02<00:00,  6.16s/it]
player stats passing : done
page 53: 100%|██████████| 53/53 [04:41<00:00,  5.32s/it]
player stats summary : done
page 53: 100%|██████████| 53/53 [04:58<00:00,  5.63s/it]
player stats defensive : done
page 53: 100%|██████████| 53/53 [06:16<00:00,  7.11s/it]
player stats offensive : done
page 53: 100%|██████████| 53/53 [05:09<00:00,  5.84s/it]
player stats passing : done
page 54: 100%|██████████| 54/54 [04:43<00:00,  5.25s/it]
player stats summary : done
page 54: 100%|██████████| 54/54 [05:37<00:00,  6.25s/it]
player stats defensive : done
page 54: 100%|██████████| 54/54 [07:25<00:00,  8.25s/it]
player stats offensive : done
page 22:  41%|████      | 22/54 [02:38<04:01,  7.55s/